## SQL queries (RUN LOCALLY ONLY)

### These SQL queries are based on the requirements of the Data analytics team: 

1. List the most active users
2. List longest open event (for Issue from started_at to closed_at for PullRequest from created_at to merged_at )
3. List the most popular five tags for all repositories (or label for Issue)
4. List the total completed event count per repository for a given period
5. List top users based on number of repositories they contributed

##TODO : MAKE THIS RUN IN CONTAINER USING DOCKERFILE

In [ ]:
# Importing the .env variables 

import os
from dotenv import load_dotenv

# Path to the .env file
env_path = '.\.\.env'

# Load the environment variables from the .env file
load_dotenv(dotenv_path=env_path)

# Access the environment variables
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')
TABLE_NAME = os.getenv('TABLE_NAME')


In [ ]:
#Initialising the connection to the MySQL database
import mysql.connector

cnx = mysql.connector.connect(
    user=DB_USER,
    port=DB_PORT,
    password=DB_PASSWORD,
    host='localhost',
    database=DB_NAME
)

cursor = cnx.cursor()

### Queries as per the requirements


In [ ]:
# List the most active users
query1 = f"""
SELECT user, COUNT(*) AS event_count
FROM {TABLE_NAME}
GROUP BY user
ORDER BY event_count DESC
LIMIT 10;
"""

cursor.execute(query1)
result1 = cursor.fetchall()
print(result1)


In [ ]:
query2 = F"""
SELECT id, title, user, repository_name, repository_owner, created_at, merged_at, TIMESTAMPDIFF(day, created_at, COALESCE(merged_at, NOW())) AS days_open 
FROM {TABLE_NAME}
WHERE state = 'open' OR (state = 'closed' AND merged_at BETWEEN 'start_date' AND 'end_date')
ORDER BY days_open DESC
LIMIT 5;
"""

cursor.execute(query2)
result2 = cursor.fetchall()
print(result2)



In [ ]:
# List the most popular five tags for all repositories
query3 = f"""
SELECT tags, COUNT(*) AS tag_count
FROM {TABLE_NAME}
WHERE tags IS NOT NULL
GROUP BY tags
ORDER BY tag_count DESC
LIMIT 5;
"""

cursor.execute(query3)
result3 = cursor.fetchall()
print(result3)

In [ ]:
# List the total completed event count per repository for a given period
start_date = '2022-01-01'
end_date = '2022-12-31'

query4 = f"""
SELECT repository_name, COUNT(*) AS completed_event_count
FROM {TABLE_NAME}
WHERE state = 'closed' AND merged_at BETWEEN '{start_date}' AND '{end_date}'
GROUP BY repository_name;
"""

cursor.execute(query4)
result4 = cursor.fetchall()
print(result4)


In [ ]:
# List top users based on number of repositories they contributed
query5 = f"""
SELECT user, COUNT(DISTINCT repository_name) AS repository_count
FROM {TABLE_NAME}
GROUP BY user
ORDER BY repository_count DESC
LIMIT 10;
"""

cursor.execute(query5)
result5 = cursor.fetchall()
print(result5)


In [ ]:
# close the connection
cnx.close()